In [1]:
import pandas as pd
import pickle

In [2]:
import os

DATA_DIR = 'data/train-scene classification'
MODEL_DIR = "saved_model/"

IMG_DIR = os.path.join(DATA_DIR, 'train')

TRAIN_CSV = os.path.join(DATA_DIR, 'train.csv')
TEST_CSV = "data/test_WyRytb0.csv"

RESULTS = "results/"

image = 'image_name'
label = 'label'

numpy_dir = "numpy_outs/"

H = W = 150
ORIG_H = ORIG_W = 150

In [3]:
# from utlities import stratified_split
#generate metadata
# stratified_split(TRAIN_CSV,"k_fold_metadata")

In [4]:
test_data = pd.read_csv(TEST_CSV)

In [5]:
from PIL import Image
img = Image.open("data/train-scene classification/train/"+"2.jpg")
img.size

(150, 150)

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torchvision.transforms.functional as Func
from torch.utils import data
from torchvision import transforms, utils

import tqdm
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

class AVDataset(data.Dataset):
    def __init__(self, IMG_DIR, file_list, is_test = False, transform=None):
        self.is_test = is_test
        self.root_path = IMG_DIR
        self.file_list = file_list
        self.transform = transform
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        if index not in range(0, len(self.file_list)):
            return self.__getitem__(np.random.randint(0, self.__len__()))
        
        file_id = self.file_list[index]
        
        image_path = os.path.join(self.root_path, file_id[0])
        
        image = load_image(image_path)
        if self.transform:
            image = self.transform(image)
            
        if self.is_test==True:
            return image
        else:
            return image,file_id[1]

In [7]:
no_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
                                   ])
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])
train_transforms2 = transforms.Compose([
    transforms.ColorJitter(brightness=0.4, contrast=0.2, saturation=0.5, hue=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])
train_transforms3 = transforms.Compose([
    transforms.RandomHorizontalFlip(p=1.0),
    transforms.ColorJitter(brightness=0.4, contrast=0.2, saturation=0.5, hue=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

In [8]:
%matplotlib inline
import cv2
from PIL import Image
import matplotlib.pyplot as plt
def load_image(path):
    img = cv2.imread(str(path))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    if img.shape != (H,W,3):
        img = cv2.resize(img, (H,W), interpolation = cv2.INTER_AREA)
    return Image.fromarray(img)

In [9]:
def create_file_list(dataframe):
    return [tuple(x) for x in dataframe.values]

In [10]:
import torch
from torch import nn
from pathlib import Path
from torch.nn import functional as F
from torchvision import models
from torchvision.transforms import ToTensor, Normalize, Compose
import cv2
from pathlib import Path

In [11]:
from cnn_finetune import make_model
model = make_model('resnet34', num_classes=6, pretrained=True, input_size=(150, 150))


In [12]:
import torch
from torchvision import models
from torchsummary import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vgg = model.to(device)


# summary(vgg, (3, 150, 150))

In [13]:
def save_checkpoint(state, is_best,filename):
#     filename='saved_model/best_acc.pth.tar'
    """Save checkpoint if a new best is achieved"""
    if is_best:
        print ("=> Saving a new best")
        torch.save(state, filename)  # save checkpoint
    else:
        print ("=> Validation loss did not improve")

In [14]:
from sklearn.metrics import accuracy_score, classification_report , confusion_matrix
def train_data(train_dataloader,val_dataloader, filename):
    best_acc=0.0
    model = make_model('resnet50', num_classes=6, pretrained=True, input_size=(150, 150), dropout_p=0.5)
    model = model.to(device)
    learning_rate = 1e-3
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9)
    criterion =  nn.CrossEntropyLoss()
    epoch=10
    for e in range(epoch):
    #     learning_rate = cyclic_lr(e)
        print("learning_rate",learning_rate)
        train_loss = []
        train_preds = []
        train_truth = []
        for image, mask in tqdm.tqdm(train_dataloader):

            optimizer.zero_grad()

            image = image.type(torch.FloatTensor).cuda()
            y_pred = model(Variable(image))
            loss = criterion(y_pred, Variable(mask.cuda()))

            _,predicted = torch.max(y_pred,1)
            train_preds.extend(predicted.cpu().data.numpy())
            train_truth.extend(mask.cpu().data.numpy())

            loss.backward()

            optimizer.step()
            train_loss.append(loss.item())

        val_loss = []
        val_preds = []
        val_truth = []
        for image, mask in val_dataloader:
            image = image.type(torch.FloatTensor).cuda()
            y_pred = model(Variable(image))

            _,predicted = torch.max(y_pred,1)
            val_preds.extend(predicted.cpu().data.numpy())
            val_truth.extend(mask.cpu().data.numpy())

            loss = criterion(y_pred, Variable(mask.cuda()))
            val_loss.append(loss.item())
        
        train_acc = accuracy_score(train_truth,train_preds)
        val_acc = accuracy_score(val_truth,val_preds)
        print("Epoch: %d, Train loss: %.3f,  Val loss: %.3f" % (e, np.mean(train_loss), np.mean(val_loss)))
        print("Train acc: %.3f, Val acc: %.3f" %(train_acc, val_acc))
#         print("Classification report" , classification_report(val_truth, val_preds))
        print("Confusion report" , confusion_matrix(val_truth, val_preds))
        
        is_best = bool(val_acc>best_acc)
        best_acc = max(val_acc,best_acc)
        save_checkpoint({
            'epoch': e + 1,
            'state_dict': model.state_dict(),
            'best_acc': best_acc
        }, is_best, filename)

In [15]:
for i in range(0,10):
    print("*"*20, "Fold", i,"*"*10)
    train = pd.read_csv("k_fold_metadata/train_{}.csv".format(i))
    val = pd.read_csv("k_fold_metadata/val_{}.csv".format(i))
    file_list_train = create_file_list(train)
    file_list_val = create_file_list(val)
    dataset = torch.utils.data.ConcatDataset([AVDataset(IMG_DIR, file_list_train, transform=train_transforms),
                                              AVDataset(IMG_DIR, file_list_train, transform=no_transforms),
                                              AVDataset(IMG_DIR, file_list_train, transform=train_transforms2),
                                              AVDataset(IMG_DIR, file_list_train, transform=train_transforms3)])
    val_dataset = torch.utils.data.ConcatDataset([AVDataset(IMG_DIR, file_list_val,transform=no_transforms),
                                                  AVDataset(IMG_DIR, file_list_val,transform=train_transforms),
                                                  AVDataset(IMG_DIR, file_list_val,transform=train_transforms2),
                                                  AVDataset(IMG_DIR, file_list_val,transform=train_transforms3)])
    train_dataloader = data.DataLoader(dataset, batch_size = 16,shuffle=True, num_workers=8)
    val_dataloader = data.DataLoader(val_dataset, batch_size = 8, shuffle = False, num_workers=4)
    train_data(train_dataloader, val_dataloader, "saved_model/resnet50/model_{}.pth.tar".format(i))

******************** Fold 0 **********


  0%|          | 0/3449 [00:00<?, ?it/s]

learning_rate 0.001


  0%|          | 0/3449 [00:00<?, ?it/s]

Epoch: 0, Train loss: 0.255,  Val loss: 0.391
Train acc: 0.910, Val acc: 0.883
Confusion report [[767   6   5   4  13 153]
 [  1 959   9   7   1  11]
 [  0   9 950  68  36   5]
 [  1  17 187 875  13   3]
 [  8   5  56  34 894   7]
 [ 33  10   7   0   7 983]]
=> Saving a new best
learning_rate 0.001


...

Epoch: 7, Train loss: 0.013,  Val loss: 0.470
Train acc: 0.996, Val acc: 0.906
Confusion report [[857   2   2   4  18  61]
 [  0 977   5   2   0   4]
 [  5   3 877 118  53   8]
 [  5   6 120 910  45   6]
 [ 10   3  19  28 932   8]
 [ 34   2   0   3   1 996]]
=> Validation loss did not improve
learning_rate 0.001


  0%|          | 0/3450 [00:00<?, ?it/s]

Epoch: 8, Train loss: 0.011,  Val loss: 0.451
Train acc: 0.996, Val acc: 0.915
Confusion report [[885   0   1   9  11  38]
 [  0 976   2   6   0   4]
 [  4   6 906 100  45   3]
 [  2   7 116 940  19   8]
 [  8   3  28  22 935   4]
 [ 62  12   2   0   1 959]]
=> Validation loss did not improve
learning_rate 0.001


100%|██████████| 3450/3450 [15:25<00:00,  3.73it/s]


Epoch: 9, Train loss: 0.008,  Val loss: 0.464
Train acc: 0.997, Val acc: 0.915
Confusion report [[854   1   1  12  11  65]
 [  0 979   0   5   0   4]
 [  1   7 929  95  25   7]
 [  0   8 127 938   9  10]
 [  9   4  31  38 911   7]
 [ 34   8   1   1   1 991]]
=> Validation loss did not improve


In [15]:
test_file_list = create_file_list(test_data)
print(len(test_file_list))

7301


In [16]:
def predict_model(model, weight_path, dataloader_test, result_path, fname):
    submission=test_data
    all_predictions = []
    numpy_preds = []
    checkpoint = torch.load(weight_path)
    start_epoch = checkpoint['epoch']
    best_accuracy = checkpoint['best_acc']
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    for image in tqdm.tqdm(dataloader_test):
        image = image.type(torch.FloatTensor).cuda()
        y_pred = F.softmax(model(Variable(image)))
        _,predicted = torch.max(y_pred,1)
        all_predictions.extend(predicted.cpu().data.numpy())
        numpy_preds.extend(y_pred.cpu().data.numpy())
    with open(numpy_dir+"{}{}_{}.pkl".format(result_path,fname,i), 'wb') as f:
        pickle.dump(numpy_preds, f)
    submission["label"]=all_predictions
    submission.to_csv(RESULTS+"{}{}_{}.csv".format(result_path,fname,i),index=False)

In [ ]:
model_name = "resnet34"
model = make_model(model_name, num_classes=6, pretrained=True, input_size=(150, 150))#, dropout_p=0.5)
model = model.to("cuda")
saved_folders = "{}/".format(model_name)#, "resnet50/", "se_resnet50/"]:
for trans, name in zip([no_transforms, train_transforms, train_transforms2, train_transforms3],
             ["no_transforms", "train_transforms", "train_transforms2", "train_transforms3"]):
    test_dataset = AVDataset(IMG_DIR, test_file_list, is_test = True, transform=trans)
    dataloader_test = data.DataLoader(test_dataset, batch_size = 16)
    for i,weights in enumerate(os.listdir(MODEL_DIR+saved_folders)):
        predict_model(model, MODEL_DIR+saved_folders+weights, dataloader_test, saved_folders, name)

  0%|          | 0/457 [00:00<?, ?it/s]/home/mukesh/miniconda3/envs/tgs_torch/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
 30%|██▉       | 137/457 [00:28<01:06,  4.81it/s]